### Test de calidad de respuestas a 4bits (integración y usabilidad)

### Si vas justo de espacio alomejor se tiene que ejecutar a tandadas (como me pasó a mi)

### Inicializamos el nemo

In [1]:
import sys

# importamos el modulo de funciones
sys.path.append("../..")

In [2]:
from backend.modelo_ia.vector_store import VectoreStore
from backend.modelo_ia.generic_llm import LLM
from backend.modelo_ia.nemo_core import NemoCore
from backend.modelo_ia.nemo_config import NemoConfig

/home/jorge/miniconda3/envs/AIWantTheJob/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
llm = LLM(loading_mode="No cuant")
config = NemoConfig("meta-llama/Llama-2-7b-chat-hf")

No cuant False


/home/jorge/miniconda3/envs/AIWantTheJob/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.04s/it]


In [ ]:
vs = VectoreStore()

In [4]:
Nemo = NemoCore(vs, config, llm)

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 77961.04it/s]


### Cargamos las preguntas del DataFrame (topics)

In [4]:
import time

In [5]:
import pandas as pd

df_preg = pd.DataFrame(columns=["accion", "prompt", "respuesta", "cuant", "tiempo"])
df_res = pd.DataFrame(columns=["accion", "prompt", "respuesta", "cuant", "tiempo"])
df_eval = pd.DataFrame(columns=["accion", "prompt", "respuesta", "cuant", "tiempo"])

### Generamos preguntas

In [6]:
topicos = [
    "Bee keeping",
    "Yoga monitoring",
    "Medicine expecialised in endocrinology",
    "Data Science",
    "Prdoduct Managing for a Coding Department",
]

In [8]:
preguntas = []
tiempos = []
for topico in topicos:
    t1 = time.time()
    answer = await Nemo.processCall(topico, "question")
    t2 = time.time()
    preguntas.append(answer["content"])
    tiempos.append(t2-t1)

Bee keeping
Some general context you may use is: []\The specific context is: []
You are an experienced job interviewer. Your task is to generate a single, relevant interview question based on the given topic or job position. The question should be thought-provoking and designed to assess the candidate's skills, experience, or fit for the role. Provide only the question with the minimal human small talk commentary or explanation. Bee keeping
Some general context you may use is: []\The specific context is: []
Res: {'role': 'assistant', 'content': "  Great! Here's an interview question related to Bee Keeping:\n\nQuestion: Can you describe a time when you encountered a challenging situation in your beehive, and how did you go about resolving it?"}
Yoga monitoring
Some general context you may use is: []\The specific context is: []
You are an experienced job interviewer. Your task is to generate a single, relevant interview question based on the given topic or job position. The question shou

In [9]:
preguntas, tiempos

(["  Great! Here's an interview question related to Bee Keeping:\n\nQuestion: Can you describe a time when you encountered a challenging situation in your beehive, and how did you go about resolving it?",
  '  Great! Here is a sample interview question based on the topic of yoga monitoring:\n\nQuestion: Can you describe a time when you had to monitor the progress of multiple clients simultaneously in their yoga practices? How did you ensure that each client received personalized attention while also meeting your deadlines and maintaining quality standards? (Minimal Human Small Talk Commentary)',
  '  Great! For a Medicine specialization in Endocrinology, here\'s a thoughtful interview question:\n\nQuestion: "Can you share an example of how you have worked effectively with patients who require complex medication regimens due to chronic hormone imbalances? How do you ensure these patients adhere to their treatment plans while minimizing potential side effects?"',
  "  Great! Here's an in

In [10]:
df_preg["accion"] = ["generar pregunta" for i in range(len(topicos))]
df_preg["prompt"] = topicos
df_preg["respuesta"] = preguntas
df_preg["cuant"] = ["No Cuantization" for i in range(len(topicos))]
df_preg["tiempo"] = tiempos

In [11]:
df_preg

,accion,prompt,respuesta,cuant,tiempo
0,generar pregunta,Bee keeping,Great! Here's an interview question related ...,No Cuantization,134.743522
1,generar pregunta,Yoga monitoring,Great! Here is a sample interview question b...,No Cuantization,284.228114
2,generar pregunta,Medicine expecialised in endocrinology,Great! For a Medicine specialization in Endo...,No Cuantization,259.974675
3,generar pregunta,Data Science,Great! Here's an interview question for a Da...,No Cuantization,264.594889
4,generar pregunta,Prdoduct Managing for a Coding Department,Great! Here is an interview question for a P...,No Cuantization,251.600534


In [12]:
df_preg.to_csv("NoCuantpreguntas.csv", index=False)

### Generamos respuestas

### En mi ordenador consistenemente usando Nemo se pasa de memoria, así que se peueba con llm, Introduce un poco de BIAS de tiempo (es mas rapido que hacerlo desde el Nemo)

In [7]:
preguntas_res = [
  '  Great! Here is an interview question related to beekeeping:\n\nQuestion: Can you describe a time when you had to navigate a complex issue in your apiary? How did you approach the problem, and what was the outcome?',
  '  Great, thank you! For a Yoga Monitoring position, here is a thought-provoking interview question:\n\n"Can you share an example of how you have monitored and interpreted data from yoga sessions in the past? How did you ensure that the information was accurate and actionable for both practitioners and coaches?"',
  '  Great, thank you for letting me know! Here\'s a sample interview question for an Endocrinologist position:\n\n"Can you tell me about a time when you had to navigate a complex case of hormone imbalance in a patient? How did you manage to identify the root cause and develop an effective treatment plan?"',
  '  Great! Here\'s an interview question for a Data Science position:\n\nQuestion: "Can you describe a situation where you had to navigate a complex data set without clear documentation or structure? How did you approach the problem, and what was the outcome?"',
  '  Question:\n\n"Can you give an example of how you would balance technical expertise and customer needs when managing a team of software developers? How do you ensure that the products developed by your team meet the highest quality standards while also addressing market demand?"'
  ]

In [7]:
respuestas_res = []
tiempos_res = []
for res in preguntas_res:
    t1 = time.time()
    answer = await llm.emulate_answer(res)
    t2 = time.time()
    respuestas_res.append(answer)
    tiempos_res.append(t2-t1)

You are a job candidate in an interview. Your task is to provide a concise, professional answer to the given interview question. Your response should demonstrate relevant skills, experience, or qualities that would make you a strong candidate for the position. Answer as a human would, with a balance of confidence and humility. Provide only the answer without any additional commentary.   Great! Here is an interview question related to beekeeping:

Question: Can you describe a time when you had to navigate a complex issue in your apiary? How did you approach the problem, and what was the outcome?
You are a job candidate in an interview. Your task is to provide a concise, professional answer to the given interview question. Your response should demonstrate relevant skills, experience, or qualities that would make you a strong candidate for the position. Answer as a human would, with a balance of confidence and humility. Provide only the answer without any additional commentary.   Great, t

In [8]:
respuestas_res, tiempos_res

(['  Absolutely! I have extensive experience managing my own apinews network and dealing with various complicated challenges therein. Once, one of our hives experienced a severe case of varroa mite infestation leading to diminished colony productivity; furthermore, excess moisture caused wax debris buildup within frames. To address this matter effectively—and after analyzing available options carefully* —I decided on integrated pest management strategies involving treatments and queen exclusion techniques simultaneously reducing mitescould compromise vital brood development areas. The implementation yielded satisfactory results over two harvest seasons due largelyto early detections allowing us quick adaptation before inflictive outcomes materialized via loss of pollinators essential for bountiful crop production!',
  '  Of course! In my previous role as a Yoga Instructor at XYZ Wellness Center, I was responsible for monitoring and interpreting data from various yoga classes throughout

In [10]:
df_res["accion"] = ["generar respuesta" for i in range(len(tiempos_res))]
df_res["prompt"] = preguntas_res
df_res["respuesta"] = respuestas_res
df_res["cuant"] = ["No Cuantization" for i in range(len(tiempos_res))]
df_res["tiempo"] = tiempos_res

In [11]:
df_res

,accion,prompt,respuesta,cuant,tiempo
0,generar respuesta,Great! Here is an interview question related...,Absolutely! I have extensive experience mana...,No Cuantization,361.028187
1,generar respuesta,"Great, thank you! For a Yoga Monitoring posi...",Of course! In my previous role as a Yoga Ins...,No Cuantization,310.549371
2,generar respuesta,"Great, thank you for letting me know! Here's...",Of course! As an experienced endocrinologist...,No Cuantization,768.757888
3,generar respuesta,Great! Here's an interview question for a Da...,Sure! In my previous role as a Data Scientis...,No Cuantization,817.058042
4,generar respuesta,"Question:\n\n""Can you give an example of how...",Absolutely! In my current role at XYZ Compan...,No Cuantization,1546.323392


In [12]:
df_res.to_csv("NoCuantrespuestas.csv", index=False)

### Generamos evaluaciones

### Doble truco, usar LLM y recortar la uiltima pregunta para evitar desbordamientos

In [8]:
respuestas = [
    "Question: " + preguntas_res[0] + "\nAnswer: " + "I dont want to ", # se niega
    "Question: " + preguntas_res[1] + "\nAnswer: " + "I dont know", # no sabe
    "Question: " + preguntas_res[2] + "\nAnswer: " + "Medicine can be used for doing X and Y", # on topic pero muy poco
    # respuestas del propio bot
    "Question: " + preguntas_res[3] + "\nAnswer: " + "Absolutely! Here's my answer:\n\nIn my previous role at XYZ Enterprises, I faced a challenging situation where I had to navigate a large and complex dataset without proper documentation or structure. The dataset consisted of various types of files, including CSV, JSON, and Excel sheets, and it was collected from different sources over several years. Despite the lack of clear documentation, I was able to successfully navigate the data by following a systematic approach.\n\nFirstly, I started by organizing the datasets alphabetically based on the file names, which helped me identify the main categories of data. Next, I used metadata to link the various files together and created a simplified naming convention to facilitate easy identification. This allowed me to gain insights into the structure and content of each file, which ultimately led to the development of a visual representation using Tableau software.\n\nThrough this process, I identified key trends and patterns within the data sets, such as customer preferences and buying behaviors, which were not explicitly documented. By visualizing these patterns, I could communicate my findings effectively to stakeholders across the organization, resulting in informative decisions that improved overall business performance.\n\nThrough this experience, I learned the importance of staying proactive, creative, and patient when dealing with complex data sets, especially without proper documentation or guidance. I believe my ability to work independently, think critically, and deliver actionable results will serve me well in this role as a Data Scientist",
    "Question: " + preguntas_res[4] + "\nAnswer: " + "Of course! Here's my response:\n\nAs a seasoned technology leader with over a decade of experience managing software development teams, I have consistently demonstrated my ability to strike a delicate balance between technical expertise and customer needs. To achieve this, I employ several strategies:\n\n1. Prioritize clear communication: Through regular check-ins and progress updates, I maintain open lines of communication with both my technical and non-technical stakeholders. By ensuring everyone is on the same page, we can align on project goals and objectives while continuously soliciting feedback to iterate and improve upon our solutions.\n2. Foster a culture of collaboration: Encouraging collaboration among my team members allows us to pool diverse perspectives and leverage each person's strengths. This approach not only fosters innovation but also helps to create more robust and well-rounded product offerings that cater to evolving market demands.\n3. Lead by example: As a hands-on manager, I lead projects side-by-side with my team rather than relegating myself solely to a leadership role. By remaining deeply involved in the development process, I set the bar high for technical excellence while modeling collaborative behavior for my team.\n4. Implement rigorous quality assurance processes: Before launching new products or features, we conduct thorough QA testing to ensure they adhere to industry best practices and client expectations. Conducting periodic code reviews and security audits maintains quality standards throughout the entire product lifecycle."
]

In [9]:
evaliuaciones = []
tiempos_eval = []
for res in respuestas:
    t1 = time.time()
    answer = await llm.punctuate_answer(res)
    t2 = time.time()
    evaliuaciones.append(answer)
    tiempos_eval.append(t2-t1)

You are an expert hiring manager evaluating candidates' responses to interview questions. Your task is to assess the given answer based on its relevance, accuracy, and effectiveness. Use the following scale for evaluation:

0 - Refuses to answer
1 - Poor: Answer is off-topic or inadequate
2 - Fair: Answer is somewhat relevant but lacks depth or clarity
3 - Good: Answer is relevant and demonstrates basic understanding
4 - Very Good: Answer is well-articulated and shows strong relevant skills or experience
5 - Excellent: Answer is exceptional, demonstrating deep understanding and outstanding fit for the role

Provide your evaluation in the following format:
Score: [0-5]
Brief explanation: [Your concise assessment of the answer's strengths and weaknesses with humanlike writting]

 Question:   Great! Here is an interview question related to beekeeping:

Question: Can you describe a time when you had to navigate a complex issue in your apiary? How did you approach the problem, and what was 

In [10]:
evaliuaciones, tiempos_eval

(['  Score: 1\nBrief explanation: The candidate refused to answer the question, which indicates that they may not have any experiences to draw from or may feel uncomfortable discussing their abilities in this area. Lack of relevant information makes it difficult to evaluate their potential to handle complex issues in an apiary.',
  '  Score: 0\n\nExplanation: Unfortunately, this candidate refused to answer the question directly. They were unable or unwilling to provide any examples of their previous experience monitoring and interpreting data from yoga sessions, which is crucial for this role as a Yoga Monitoring professional. This lack of response makes it challenging for me to evaluate their qualifications and potential contributions to the team.',
  '  Score: 1\n\nBrief explanation: The candidate provides little context or specific examples related to endocrine disorders, instead discussing the use of medicine in general terms. This response does not demonstrate any depth of knowled

In [11]:
df_eval["accion"] = ["generar evaluacion" for i in range(len(topicos))]
df_eval["prompt"] = respuestas
df_eval["respuesta"] = evaliuaciones
df_eval["cuant"] = ["No Cuantization" for i in range(len(topicos))]
df_eval["tiempo"] = tiempos_eval

In [12]:
df_eval

,accion,prompt,respuesta,cuant,tiempo
0,generar evaluacion,Question: Great! Here is an interview questi...,Score: 1\nBrief explanation: The candidate r...,No Cuantization,149.810956
1,generar evaluacion,"Question: Great, thank you! For a Yoga Monit...","Score: 0\n\nExplanation: Unfortunately, this...",No Cuantization,173.730048
2,generar evaluacion,"Question: Great, thank you for letting me kn...",Score: 1\n\nBrief explanation: The candidate...,No Cuantization,210.121716
3,generar evaluacion,Question: Great! Here's an interview questio...,Score: 4 (Very Good)\n\nExplanation:\nThe ca...,No Cuantization,452.266548
4,generar evaluacion,"Question: Question:\n\n""Can you give an exam...",Score: 4 (Very good)\n\nExplanation: The can...,No Cuantization,335.527383


In [13]:
df_eval.to_csv("NoCuantEvals.csv", index=False)

## El test de Integración se pasa, se  adhieren al contexto según lo esperado

Aunque tengan tendencias todavía de LLM (por ejemplo empezar con "Thought Provoking")